In [ ]:
def mul(x,y):
    res = []
    for i in x:
        if i+"'" in y or (len(i)==2 and i[0] in y):
            return []
        else:
            res.append(i)
    for i in y:
        if i not in res:
            res.append(i)
    return res

def multiplica(x,y):
    res = []
    for i in x:
        for j in y:
            tmp = multiplica(i,j)
            res.append(tmp) if len(tmp) != 0 else None
    return res

def refinar(my_list,dc_list):
    res = []
    for i in my_list:
        if int(i) not in dc_list:
            res.append(i)
    return res

def BuscarIPE(x):
    res = []
    for i in x:
        if len(x[i]) == 1:
            res.append(x[i][0]) if x[i][0] not in res else None
    return res

def BuscarVariables(x):
    var_list = []
    for i in range(len(x)):
        if x[i] == '0':
            var_list.append(chr(i+65)+"'")
        elif x[i] == '1':
            var_list.append(chr(i+65))
    return var_list

def recorta(x):
    elementos_recortados = []
    for i in x:
        elementos_recortados.extend(x[i])
    return elementos_recortados

def buscaMinterminos(a):
    gaps = a.count('-')
    if gaps == 0:
        return [str(int(a,2))]
    x = [bin(i)[2:].zfill(gaps) for i in range(pow(2,gaps))]
    temp = []
    for i in range(pow(2,gaps)):
        temp2,ind = a[:],-1
        for j in x[0]:
            if ind != -1:
                ind = ind+temp2[ind+1:].find('-')+1
            else:
                ind = temp2[ind+1:].find('-')
            temp2 = temp2[:ind]+j+temp2[ind+1:]
        temp.append(str(int(temp2,2)))
        x.pop(0)
    return temp

def compara(a,b):
    c = 0
    for i in range(len(a)):
        if a[i] != b[i]:
            mismatch_index = i
            c += 1
            if c>1:
                return (False,None)
    return (True,mismatch_index)

def remueveTerminos(_chart,terms):
    for i in terms:
        for j in buscaMinterminos(i):
            try:
                del _chart[j]
            except KeyError:
                pass
print('Por favor ingrese los términos separados por un espacio')
mt = [int(i) for i in input("Ingrese los mintérminos: ").strip().split()]
dc = [int(i) for i in input("Ingrese los don't care (Si existen): ").strip().split()]
mt.sort()
minterminos = mt+dc
minterminos.sort()
size = len(bin(minterminos[-1]))-2
grupos,all_pi = {},set()

for minterm in minterminos:
    try:
        grupos[bin(minterm).count('1')].append(bin(minterm)[2:].zfill(size))
    except KeyError:
        grupos[bin(minterm).count('1')] = [bin(minterm)[2:].zfill(size)]

print("\n\n\n\nNúmero de Gpo.\tMintérminos\t\tExpresión en BCD\n%s"%('='*60))
for i in sorted(grupos.keys()):
    print("%5d:"%i)
    for j in grupos[i]:
        print("\t\t    %-20d%s"%(int(j,2),j))
    print('-'*60)

while True:
    tmp = grupos.copy()
    grupos,m,marcados,debo_parar = {},0,set(),True
    l = sorted(list(tmp.keys()))
    for i in range(len(l)-1):
        for j in tmp[l[i]]:
            for k in tmp[l[i+1]]:
                res = compara(j,k)
                if res[0]:
                    try:
                        grupos[m].append(j[:res[1]]+'-'+j[res[1]+1:]) if j[:res[1]]+'-'+j[res[1]+1:] not in grupos[m] else None
                    except KeyError:
                        grupos[m] = [j[:res[1]]+'-'+j[res[1]+1:]]
                    debo_parar = False
                    marcados.add(j)
                    marcados.add(k)
        m += 1
    desmarcados_local = set(recorta(tmp)).difference(marcados)
    all_pi = all_pi.union(desmarcados_local)
    print("Elementos desmarcados(Implicantes Primos) de la tabla:",None if len(desmarcados_local)==0 else ', '.join(desmarcados_local))
    if debo_parar:
        print("\n\nAll Implicantes Primos: ",None if len(all_pi)==0 else ', '.join(all_pi))
        break

    print("\n\n\n\nNúmero de Gpo\tMintérminos\t\tExpresión en BCD\n%s"%('='*60))
    for i in sorted(grupos.keys()):
        print("%5d:"%i)
        for j in grupos[i]:
            print("\t\t%-24s%s"%(','.join(buscaMinterminos(j)),j))
        print('-'*60)

sz = len(str(mt[-1]))
chart = {}
print('\n\n\nImpresión de los implicantes primos escenciales:\n\n  Mintérminos  |%s\n%s'%(' '.join((' '*(sz-len(str(i))))+str(i)
for i in mt),'='*(len(mt)*(sz+1)+16)))
for i in all_pi:
    minterminos_mezclados,y = buscaMinterminos(i),0
    print("%-16s|"%','.join(minterminos_mezclados),end='')
    for j in refinar(minterminos_mezclados,dc):
        x = mt.index(int(j))*(sz+1)
        print(' '*abs(x-y)+' '*(sz-1)+'X',end='')
        y = x+sz
        try:
            chart[j].append(i) if i not in chart[j] else None
        except KeyError:
            chart[j] = [i]
    print('\n'+'-'*(len(mt)*(sz+1)+16))


IPE = BuscarIPE(chart)
print("\nImplicantes Primos Escenciales: "+', '.join(str(i) for i in IPE))
remueveTerminos(chart,IPE)

if(len(chart) == 0):
    resultado_final = [BuscarVariables(i) for i in IPE]
else:
    P = [[BuscarVariables(j) for j in chart[i]] for i in chart]
    while len(P)>1:
        P[1] = multiplica(P[0],P[1])
        P.pop(0)
    resultado_final = [min(P[0],key=len)]
    resultado_final.extend(BuscarVariables(i) for i in IPE)
print('\nSolución: F = '+' + '.join(''.join(i) for i in resultado_final))


Por favor ingrese los términos separados por un espacio
Ingrese los mintérminos: 0 1 3 5 6 8 9 2 4 5 
Ingrese los don't care (Si existen): 




Número de Gpo.	Mintérminos		Expresión en BCD
    0:
		    0                   0000
------------------------------------------------------------
    1:
		    1                   0001
		    2                   0010
		    4                   0100
		    8                   1000
------------------------------------------------------------
    2:
		    3                   0011
		    5                   0101
		    5                   0101
		    6                   0110
		    9                   1001
------------------------------------------------------------
Elementos desmarcados(Implicantes Primos) de la tabla: None




Número de Gpo	Mintérminos		Expresión en BCD
    0:
		0,1                     000-
		0,2                     00-0
		0,4                     0-00
		0,8                     -000
----------------------------------------------------------